In [1]:
file_dir='../input/entity-annotated-corpus/ner_dataset.csv'
target='Tag'
drop_cols=['Sentence #','POS']
word_col='Word'

USE_BERT=False
USE_W2V=True

In [2]:
!pip install pycaret

     |████████████████████████████████| 264 kB 892 kB/s 
     |████████████████████████████████| 167 kB 5.7 MB/s 
     |████████████████████████████████| 24.2 MB 439 kB/s 
     |████████████████████████████████| 14.4 MB 40.4 MB/s 
     |████████████████████████████████| 105 kB 67.0 MB/s 
     |████████████████████████████████| 25.9 MB 52.4 MB/s 
     |████████████████████████████████| 54 kB 2.2 MB/s 
     |████████████████████████████████| 79 kB 6.1 MB/s 
     |████████████████████████████████| 1.1 MB 44.7 MB/s 
     |████████████████████████████████| 1.7 MB 49.9 MB/s 
  Installing build dependencies ... - \ | / done
  Getting requirements to build wheel ... - done
  Installing backend dependencies ... - \ | done
    Preparing wheel metadata ... - done
     |████████████████████████████████| 1.7 MB 36.2 MB/s 
  Created wheel for alembic: filename=alembic-1.4.1-py2.py3-none-any.whl size=158155 sha256=85cd0dda25172a79ded00c612c4cf2321294a1b2b8887f1cde07bac85a8c8f3f
  St

In [3]:
import pandas as pd 
import numpy as np 
import sys
import gensim.downloader as api
import nltk
from gensim.models.word2vec import Word2Vec
from nltk.corpus import stopwords
from nltk.tag.stanford import StanfordNERTagger
import tensorflow as tf
import transformers
from transformers import BertTokenizer
import warnings
import sklearn
from pycaret.classification import *
from sklearn.metrics import average_precision_score
import sklearn
warnings.filterwarnings('ignore')


In [4]:
split_df=pd.read_csv(file_dir)
split_df.drop(drop_cols,inplace=True,axis=1)
split_df

,Word,Tag
0,Thousands,O
1,of,O
2,demonstrators,O
3,have,O
4,marched,O
...,...,...
1048570,they,O
1048571,responded,O
1048572,to,O
1048573,the,O


In [5]:
# pos tag
def get_pos(tokens):
    pos_tags_raw=nltk.pos_tag(tokens)
    pos_tags=list(map(lambda x:x[1],pos_tags_raw))
    return pos_tags

# stop word
def get_stop_word(tokens):
    stop_words_list=stopwords.words('english')
    stop_words=list(map(lambda x:x in stop_words_list,tokens))
    return stop_words

# NER
# https://nlp.stanford.edu/software/CRF-NER.shtml
def get_ner(tokens):
    model_path='stanford_ner\\english.muc.7class.distsim.crf.ser.gz'
    # model_path='stanford_ner\\english.all.3class.distsim.crf.ser.gz'
    jar_path='stanford_ner\\stanford-ner.jar'
    tagger = StanfordNERTagger(model_path,jar_path,encoding='utf-8')
    # tagger = StanfordNERTagger(model_path,encoding='utf-8')
    tagged = tagger.tag(tokens)
    tagged=list(map(lambda x:x[1],tagged))
    return tagged

# upper/lower case information, acronyms, punctuation marks, etc.
# % of upper case (or number)
def get_syntactic(tokens):
    return list(map(lambda x:sum(np.array(list(x.upper()))==np.array(list(x)))/len(x),tokens))

# word2vec or bert representation
def get_word2vec(tokens):
    res=[]
    for token in tokens:
        try:wt=w2v_model.wv.get_vector(token)
        except:wt=np.zeros(100)
        res.append(wt)
    return np.array(res)

# bert representation
def load_bert():
    input_ids = tf.keras.layers.Input(
        shape=(3), dtype=tf.int32, name="input_ids")

    embedding = transformers.TFBertModel.from_pretrained("cambridgeltl/BioRedditBERT-uncased")
    out = embedding(input_ids)

    model = tf.keras.models.Model(inputs=[input_ids], outputs=out[0])
    model.compile(optimizer=tf.keras.optimizers.Adam(),loss="categorical_crossentropy",metrics=["acc"],)
    return model

def get_bert(tokens):
    def tokenize_data(data):
        tokenizer = BertTokenizer.from_pretrained("cambridgeltl/BioRedditBERT-uncased")
        encoded = tokenizer.batch_encode_plus(
            data,
            add_special_tokens=True,
            max_length=3,
            return_attention_mask=True,
            return_token_type_ids=True,
            pad_to_max_length=True,
            return_tensors="tf",
        )

        return np.array(encoded["input_ids"], dtype="int32")

    token_bert=tokenize_data(tokens)
    emb=bert_model.predict(token_bert)
    return emb.mean(axis=1)

In [6]:
%%time
print('get pos tag...')
split_df['pos']=get_pos(split_df[word_col].values)

print('get stop word...')
split_df['stop_word']=get_stop_word(split_df[word_col].values)

print('get synt...')
split_df['#_of_non_lower']=split_df[word_col].transform(lambda x:sum(np.array(list(x.upper()))==np.array(list(x))))
split_df['len']=split_df[word_col].transform(lambda x:len(x))
tokens_lower=list(map(lambda x:x.lower(),split_df[word_col].values))


print('get lag...')
lag_feas=['#_of_non_lower','pos']
lags=3
for col in lag_feas:
    for lag in range(lags):
        split_df['prev_%s_%s'%(col,lag+1)]=split_df[col].shift(lag+1)
        split_df['next_%s_%s'%(col,lag+1)]=split_df[col].shift(-(lag+1))   
    
if USE_W2V:
    print('get word2vec...')
    corpus = api.load('text8') 
    w2v_model = Word2Vec(corpus)  # time consuming
    temp=get_word2vec(tokens_lower)
    for i in range(temp.shape[1]):
        split_df['word_2vec_%s'%(i+1)]=temp[:,i]

if USE_BERT:
    print('get BERT...')
    bert_model=load_bert()
    temp=get_bert(tokens)
    for i in range(temp.shape[1]):
        split_df['bert_%s'%(i+1)]=temp[:,i]

split_df.head()

get pos tag...
get stop word...
get synt...
get lag...
get word2vec...
[==================================================] 100.0% 31.6/31.6MB downloaded
CPU times: user 6min 35s, sys: 5.43 s, total: 6min 40s
Wall time: 3min 54s


,Word,Tag,pos,stop_word,#_of_non_lower,len,prev_#_of_non_lower_1,next_#_of_non_lower_1,prev_#_of_non_lower_2,next_#_of_non_lower_2,prev_#_of_non_lower_3,next_#_of_non_lower_3,prev_pos_1,next_pos_1,prev_pos_2,next_pos_2,prev_pos_3,next_pos_3,word_2vec_1,word_2vec_2,word_2vec_3,word_2vec_4,word_2vec_5,word_2vec_6,word_2vec_7,word_2vec_8,word_2vec_9,word_2vec_10,word_2vec_11,word_2vec_12,word_2vec_13,word_2vec_14,word_2vec_15,word_2vec_16,word_2vec_17,word_2vec_18,word_2vec_19,word_2vec_20,word_2vec_21,word_2vec_22,word_2vec_23,word_2vec_24,word_2vec_25,word_2vec_26,word_2vec_27,word_2vec_28,word_2vec_29,word_2vec_30,word_2vec_31,word_2vec_32,word_2vec_33,word_2vec_34,word_2vec_35,word_2vec_36,word_2vec_37,word_2vec_38,word_2vec_39,word_2vec_40,word_2vec_41,word_2vec_42,word_2vec_43,word_2vec_44,word_2vec_45,word_2vec_46,word_2vec_47,word_2vec_48,word_2vec_49,word_2vec_50,word_2vec_51,word_2vec_52,word_2vec_53,word_2vec_54,word_2vec_55,word_2vec_56,word_2vec_57,word_2vec_58,word_2vec_59,word_2vec_60,word_2vec_61,word_2vec_62,word_2vec_63,word_2vec_64,word_2vec_65,word_2vec_66,word_2vec_67,word_2vec_68,word_2vec_69,word_2vec_70,word_2vec_71,word_2vec_72,word_2vec_73,word_2vec_74,word_2vec_75,word_2vec_76,word_2vec_77,word_2vec_78,word_2vec_79,word_2vec_80,word_2vec_81,word_2vec_82,word_2vec_83,word_2vec_84,word_2vec_85,word_2vec_86,word_2vec_87,word_2vec_88,word_2vec_89,word_2vec_90,word_2vec_91,word_2vec_92,word_2vec_93,word_2vec_94,word_2vec_95,word_2vec_96,word_2vec_97,word_2vec_98,word_2vec_99,word_2vec_100
0,Thousands,O,NNS,False,1,9,NaN,0.0,NaN,0.0,NaN,0.0,NaN,IN,NaN,NNS,NaN,VBP,1.421284,0.016402,1.911949,2.714994,-0.593004,0.702406,-1.034192,-2.068728,-0.408649,0.381074,0.865582,-1.855979,-0.823347,0.467789,-1.359929,0.018515,-1.942731,-1.428731,-2.489092,-0.381786,0.660762,1.207184,-0.545124,-2.484401,-1.774102,0.598076,-1.164143,-1.470461,2.599328,-0.156253,-1.073933,1.261038,-0.321448,-0.246528,0.141310,0.289644,-0.768058,1.229070,-0.110938,-1.039593,-0.119137,1.135345,-3.822168,-0.425287,-0.202676,2.962387,0.634075,-1.478015,-0.930128,0.417163,1.882714,-0.177405,-0.769823,-0.559940,0.861612,1.282620,2.051453,1.995044,-0.017428,0.288403,0.431086,2.822564,0.189342,0.211175,-1.070597,-0.615630,-0.363360,0.446175,-2.191967,0.177563,-1.566767,-1.636885,0.441744,-0.379262,1.413376,1.127055,-0.529256,-0.095967,-1.963953,-0.536709,0.331026,-0.812682,-2.626156,1.000246,0.261747,-1.495933,1.591521,3.208377,-0.302237,-1.319445,0.800637,2.006147,-0.984088,-0.130014,-1.689246,1.100079,-3.610959,-0.156966,-5.156607,-2.026332
1,of,O,IN,True,0,2,1.0,0.0,NaN,0.0,NaN,0.0,NNS,NNS,NaN,VBP,NaN,VBN,-1.149262,-1.992989,0.035231,-0.153007,-0.639457,0.022881,-1.475076,0.509012,-0.995976,0.357413,-1.066453,1.025311,1.517382,0.047156,0.351239,0.359121,0.039140,0.516663,0.307950,-0.927842,1.621768,-0.092729,0.210708,-0.917248,-0.296736,0.484773,1.903922,0.068347,-0.021780,-1.313080,-0.319395,-0.759531,0.054108,-0.771156,-1.187336,1.194884,0.606647,1.064426,2.214024,-0.339128,0.433639,0.052514,-0.160136,-1.186648,1.352166,-1.151753,-0.609818,0.136408,0.489762,-0.034606,-0.621802,0.338283,-2.085926,1.747095,1.006143,-1.212800,0.972605,0.154178,0.646629,0.260450,1.462575,0.625859,-0.207440,-0.609215,-1.373447,-0.783509,3.014327,-0.055424,-2.375887,-2.033319,0.559523,-1.253653,0.484823,2.217039,0.947496,0.893382,0.176258,1.075231,-0.437817,1.420588,0.541593,0.299269,-0.133498,-2.166051,-1.094762,-0.431305,0.878967,-0.586833,0.940492,0.547173,0.975732,-1.034483,-0.250373,0.976415,-0.988696,-0.335740,0.371434,0.387279,-1.860047,0.009753
2,demonstrators,O,NNS,False,0,13,0.0,0.0,1.0,0.0,NaN,0.0,IN,VBP,NNS,VBN,NaN,IN,0.137477,0.230787,-0.095987,0.084007,-0.169975,-0.177495,-0.044059,0.088700,-0.078582,0.290504,0.087377,0.085510,-0.065760,-0.303749,-0.170995,0.045570,0.034340,0.158335,-0.108031,0.016771,0.202512,0.115745,-0.102669,-0.058525,-0.016854,0.014082,-0.189936,0.041302,0.243210,-0.069483,0.004008,-0.183340,-0.127313,0.296356,0.082296,0.194734,0.063

In [7]:
import pandas as pd
import numpy as np 

from pycaret.classification import *
from sklearn.metrics import average_precision_score
import sklearn

warnings.filterwarnings('ignore')

cat_feas=['pos','stop_word','len','#_of_non_lower',
          'prev_#_of_non_lower_1','prev_#_of_non_lower_2','prev_#_of_non_lower_3',
          'next_#_of_non_lower_1','next_#_of_non_lower_2','next_#_of_non_lower_3',
          'prev_pos_1','prev_pos_2','prev_pos_3',
          'next_pos_1','next_pos_2','next_pos_3']

num_feas=[x for x in split_df.columns if x not in drop_cols+cat_feas and x!=target and x!=word_col ]

target='Tag'

split_df=pd.concat((pd.get_dummies(split_df[cat_feas]),
                  split_df[num_feas+[target]]),axis=1)

org_names=split_df.columns.tolist()
split_df.columns=['col_%s'%x for x in range(len(org_names))]

target='col_%s'%org_names.index(target) 

split_df

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,col_10,col_11,col_12,col_13,col_14,col_15,col_16,col_17,col_18,col_19,col_20,col_21,col_22,col_23,col_24,col_25,col_26,col_27,col_28,col_29,col_30,col_31,col_32,col_33,col_34,col_35,col_36,col_37,col_38,col_39,col_40,col_41,col_42,col_43,col_44,col_45,col_46,col_47,col_48,col_49,col_50,col_51,col_52,col_53,col_54,col_55,col_56,col_57,col_58,col_59,col_60,col_61,col_62,col_63,col_64,col_65,col_66,col_67,col_68,col_69,col_70,col_71,col_72,col_73,col_74,col_75,col_76,col_77,col_78,col_79,col_80,col_81,col_82,col_83,col_84,col_85,col_86,col_87,col_88,col_89,col_90,col_91,col_92,col_93,col_94,col_95,col_96,col_97,col_98,col_99,col_100,col_101,col_102,col_103,col_104,col_105,col_106,col_107,col_108,col_109,col_110,col_111,col_112,col_113,col_114,col_115,col_116,col_117,col_118,col_119,col_120,col_121,col_122,col_123,col_124,col_125,col_126,col_127,col_128,col_129,col_130,col_131,col_132,col_133,col_134,col_135,col_136,col_137,col_138,col_139,col_140,col_141,col_142,col_143,col_144,col_145,col_146,col_147,col_148,col_149,col_150,col_151,col_152,col_153,col_154,col_155,col_156,col_157,col_158,col_159,col_160,col_161,col_162,col_163,col_164,col_165,col_166,col_167,col_168,col_169,col_170,col_171,col_172,col_173,col_174,col_175,col_176,col_177,col_178,col_179,col_180,col_181,col_182,col_183,col_184,col_185,col_186,col_187,col_188,col_189,col_190,col_191,col_192,col_193,col_194,col_195,col_196,col_197,col_198,col_199,col_200,col_201,col_202,col_203,col_204,col_205,col_206,col_207,col_208,col_209,col_210,col_211,col_212,col_213,col_214,col_215,col_216,col_217,col_218,col_219,col_220,col_221,col_222,col_223,col_224,col_225,col_226,col_227,col_228,col_229,col_230,col_231,col_232,col_233,col_234,col_235,col_236,col_237,col_238,col_239,col_240,col_241,col_242,col_243,col_244,col_245,col_246,col_247,col_248,col_249,col_250,col_251,col_252,col_253,col_254,col_255,col_256,col_257,col_258,col_259,col_260,col_261,col_262,col_263,col_264,col_265,col_266,col_267,col_268,col_269,col_270,col_271,col_272,col_273,col_274,col_275,col_276,col_277,col_278,col_279,col_280,col_281,col_282,col_283,col_284,col_285,col_286,col_287,col_288,col_289,col_290,col_291,col_292,col_293,col_294,col_295,col_296,col_297,col_298,col_299,col_300,col_301,col_302,col_303,col_304,col_305,col_306,col_307,col_308,col_309,col_310,col_311,col_312,col_313,col_314,col_315,col_316,col_317,col_318,col_319,col_320,col_321,col_322,col_323,col_324,col_325,col_326,col_327,col_328,col_329,col_330,col_331,col_332,col_333,col_334,col_335,col_336,col_337,col_338,col_339,col_340,col_341,col_342,col_343,col_344,col_345,col_346,col_347,col_348,col_349,col_350,col_351,col_352,col_353,col_354,col_355,col_356,col_357,col_358,col_359,col_360,col_361,col_362,col_363,col_364,col_365,col_366,col_367,col_368,col_369,col_370,col_371,col_372,col_373,col_374,col_375,col_376,col_377,col_378,col_379,col_380,col_381,col_382,col_383,col_384,col_385,col_386,col_387,col_388,col_389,col_390,col_391,col_392,col_393,col_394,col_395,col_396,col_397,col_398,col_399,col_400,col_401,col_402,col_403,col_404,col_405,col_406,col_407,col_408,col_409,col_410
0,False,9,1,NaN,NaN,NaN,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1.421284,0.016402,1.911949,2.714994,-0.593004,0.702406,-1.034192,-2.068728,-0.408649,0.381074,0.865582,-1.855979,-0.823347,0.467789,-1.359929,0.018515,-1.942731,-1.428731,-2.489092,-0

In [8]:
clf1 = setup(data = split_df.head(500000),
             target = target,
             fold=5,
             train_size=0.7,
             silent=True)

# best = compare_models(sort = 'f1',
#                       exclude = ['svm','knn','qda','nb','lda','ridge','ada'],
#                       n_select = 8) # return top 6 best models

lgb=create_model('lightgbm')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.9207,0.9219,0.3701,0.9214,0.9204,0.7168,0.7170
1,0.8703,0.7596,0.2434,0.8508,0.8592,0.4813,0.4861
2,0.9114,0.9149,0.3444,0.9129,0.9118,0.6826,0.6826
3,0.9133,0.9035,0.3425,0.9124,0.9125,0.6879,0.6880
4,0.9099,0.8875,0.3110,0.9080,0.9084,0.6682,0.6687
Mean,0.9051,0.8775,0.3223,0.9011,0.9024,0.6474,0.6485
SD,0.0178,0.0601,0.0437,0.0255,0.0220,0.0845,0.0827


In [9]:
cols=['Accuracy','Micro Precision','Micro Recall','Micro F1','Macro Precision','Macro Recall','Macro F1']
evals=[]

# for model in best:
pred_holdout = predict_model(lgb)

# evals=([ sklearn.metrics.accuracy_score(pred_holdout[target],pred_holdout['Label']),
#          sklearn.metrics.precision_score(pred_holdout[target],pred_holdout['Label'],average='micro'),
#          sklearn.metrics.recall_score(pred_holdout[target],pred_holdout['Label'],average='micro'),
#          sklearn.metrics.f1_score(pred_holdout[target],pred_holdout['Label'],average='micro'),

#          sklearn.metrics.precision_score(pred_holdout[target],pred_holdout['Label'],average='macro'),
#          sklearn.metrics.recall_score(pred_holdout[target],pred_holdout['Label'],average='macro'),
#          sklearn.metrics.f1_score(pred_holdout[target],pred_holdout['Label'],average='macro')])
# pd.DataFrame(evals,index=cols).T

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Light Gradient Boosting Machine,0.8812,0.8836,0.2588,0.8923,0.8830,0.5926,0.5945
